# TP4 - Filtrage Collaboratif

L'objectif de ce TME est d'implémenter des méthodes de filtrage collaboratif.

On utilisera les données Movie Lens 100k et Movie Lens 1M qui correspondent à deux bases de données de notes attribués par un ensemble d'utilisateurs sur des films, contenant respectivement 100 000 et 1 millions d'entrés.

On s'intéressera en premier lieu à un modèle de factorisation de matrice simple sans biais, puis on incluera dans le modèle un biais par utilisateur et par film. Enfin, on implémentera un modèle tenant compte du fait que ces biais peuvent évoluer dans le temps.

## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Chargement des données

Les fonctions pour charger les bases Movie Lens 100k et Movie Lens 1M.
On récupère un dictionnaire pour les scores et un dictionnaire pour les dates.
Le première index de ces dictionnaires est l'identifiant de l'utilisateur, et le second les films notés.

In [2]:
def loadMovieLens(path='./data100k'):
    # Get movie titles
    movies={}
    for line in open(path+'/u.item'):
        (id,title)=line.split('|')[0:2]
        movies[id]=title
    # Load data
    prefs={} # Un dictionnaire User > Item > Rating
    times={} # Un dictionnaire User > Item > Timestamps
    for line in open(path+'/u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
        times.setdefault(user,{})
        times[user][movies[movieid]]=float(ts)
    return prefs, times

In [3]:
def loadMovieLens1M(path='./data1m'):
    # Get movie titles
    movies={}
    for line in open(path+'/movies.dat'):
        id,title=line.split('::')[0:2]
        movies[id]=title
    # Load data
    prefs={}
    times={}
    for line in open(path+'/ratings.dat'):
        (user,movieid,rating,ts)=line.split('::')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
        times.setdefault(user,{})
        times[user][movies[movieid]]=float(ts)
    return prefs, times

## Représentations des données

Les matrices des scores Utilisateurs/Films sont des matrices de grandes dimensions mais sparses.
Afin de les manipuler efficacement, on emploiera 3 représentations différentes en même temps:
- Le dictionnaire des scores par utilisateurs: User > Item > Value
- Le dictionnaire des scores par films: Item > User > Value 
- La liste des triplets [User, Item, Value]

In [4]:
# Recupère une représentation des données sous la forme triplets [user, item, value] a partir d'un dictionnaire [User > item > value]
def getCouplesUsersItems(data):
    couples = []
    for u in data.keys():
        for i in data[u].keys():
            couples.append([u,i,data[u][i]])
    return couples

# Construit le dictionnaire des utilisateurs a partir des triplets [user, item, note]
def buildUsersDict(couples):
    dicUsers = {}
    for c in couples:
        if not c[0] in dicUsers.keys():
            dicUsers[c[0]] = {}
        dicUsers[c[0]][c[1]] = float(c[2])
    return dicUsers

# Construit le dictionnaire des objets a partir des triplets [user, item, note]
def buildItemsDict(couples):
    dicItems = {}
    for c in couples:
        if not c[1] in dicItems:
            dicItems[c[1]] = {}
        dicItems[c[1]][c[0]] = float(c[2])
    return dicItems

## Données de temps

Afin d'exploiter les données temporelles, on discrétise le temps en bins et on construit un vecteur qui a chaque triplets [user, item, value] associe le bins temporel correspondant.

In [5]:
def getTimeBins(couples, timedic, nbins):
    timestamps = np.zeros(len(couples))
    for i,c in enumerate(couples):
        timestamps[i] = timedic[c[0]][c[1]]
    time_bins = np.linspace(np.min(timestamps), np.max(timestamps), nbins+1)
    times = np.zeros(len(couples),int)
    for i in xrange(1,len(time_bins)):
        times = times + (timestamps > time_bins[i])
    return times

## Séparation des données en Train / Test

Pour pouvoir séparer les données en ensembles de Train et de Test, on utilisera la liste des triplets [User, Item, Scores].

In [6]:
# Split l'ensemble des triplets [user, item, note] en testProp% données de test et (1 - testProp) données de train
def splitTrainTest(couples,testProp):
    perm = np.random.permutation(couples)
    splitIndex = int(testProp * len(couples))
    return perm[splitIndex:], perm[:splitIndex]

# Modèles

## Baseline 1 : Moyenne par utilisateur

In [7]:
class baselineMeanUsers():
    def __init__(self):
        self.mean = {}
    def fit(self, dataUsers):
        self.mean = {}
        for u in dataUsers.keys():
            self.mean[u] = 0
            for i in dataUsers[u].keys():
                self.mean[u] = self.mean[u] + dataUsers[u][i]
            self.mean[u] = self.mean[u] / len(dataUsers[u])
    def predict(self, couplesTest):
        pred = np.zeros(len(couplesTest))
        for ind,c in enumerate(couplesTest):
            pred[ind] = self.mean[c[0]]
        return pred

## Baseline 2 : Moyenne par item

In [8]:
class baselineMeanItems():
    def __init__(self):            
        self.mean = {}
    def fit(self, dataItems):
        self.mean = {}
        for i in dataItems.keys():
            self.mean[i] = 0
            for u in dataItems[i].keys():
                self.mean[i] = self.mean[i] + dataItems[i][u]
            self.mean[i] = self.mean[i] / len(dataItems[i])
    def predict(self, couplesTest):
        pred = np.zeros(len(couplesTest))
        for ind,c in enumerate(couplesTest):
            pred[ind] = self.mean[c[1]]
        return pred

## Factorisation de Matrices sans biais

In [9]:
class matrixFactorisation():
    def __init__(self, k, lambd=0.2, eps=1e-5, maxIter=2000, alternate=0):
        self.k = k
        self.lambd = lambd
        self.eps = eps
        self.maxIter = maxIter
        self.alternate = alternate #Pour l'optimisation alternée: 0 si non.
    def fit(self, dataUsers, dataItems, couples):
        self.p = {}
        self.q = {}
        self.loss = []
        #Choix du paramètre a optimisé en cas d'optimisation alternée
        optimP = True
        optimQ = (self.alternate == 0)
        for i in xrange(self.maxIter):
            loss = 0
            for j in xrange(len(couples)):
                #choix d'une entrée aléatoire
                r = np.random.randint(len(couples)) 
                user = couples[r][0]
                item = couples[r][1]
                # initialisation des nouveaux vecteurs p et q
                if not user in self.p:
                    self.p[user] = np.random.rand(1,self.k)
                if not item in self.q:
                    self.q[item] = np.random.rand(self.k,1)
                # Descente de gradient
                tmp = dataUsers[user][item] - self.p[user].dot(self.q[item])[0][0]
                if (optimP):
                    self.p[user] = (1 - self.lambd * self.eps) * self.p[user] + self.eps * 2 * tmp * self.q[item].transpose()
                if (optimQ):
                    self.q[item] = (1 - self.lambd * self.eps) * self.q[item] + self.eps * 2 * tmp * self.p[user].transpose()
                loss = loss + tmp*tmp #(Sans le terme de régularisation)
            self.loss.append(loss)
            # Optimisation alternée
            if (self.alternate != 0):
                if (i % self.alternate == 0):
                    optimP = optimQ
                    optimQ = 1 - optimQ
                    print i, loss / len(couples)
            else:
                if (i % 100 == 0):
                    print i, loss / len(couples)
    def predict(self, couplesTest):
        pred = np.zeros(len(couplesTest))
        for ind,c in enumerate(couplesTest):
            pred[ind] = self.p[c[0]].dot(self.q[c[1]])[0][0]
        return pred

## Factorisation de Matrices avec biais

In [10]:
# Comme matrixFactorisation() avec plus de paramètres
class matrixFactorisationBiais():
    def __init__(self, k, lambd=0.2, eps=1e-5, maxIter=10000, alternate=0):
        self.k = k
        self.lambd = lambd
        self.eps = eps
        self.maxIter = maxIter
        self.alternate = alternate
    def fit(self, dataUsers, dataItems, couples):
        self.p = {}
        self.q = {}
        self.bu = {}
        self.bi = {}
        self.mu = np.random.random() * 2 - 1
        self.loss = []
        optimP = True
        optimQ = (self.alternate == 0)
        for i in xrange(self.maxIter):
            loss = 0
            for j in xrange(len(couples)):
                r = np.random.randint(len(couples))
                user = couples[r][0]
                item = couples[r][1]
                if not user in self.p:
                    self.p[user] = np.random.rand(1,self.k) * 2 - 1
                    self.bu[user] = np.random.rand() * 2 - 1
                if not item in self.q:
                    self.q[item] = np.random.rand(self.k,1) * 2 - 1
                    self.bi[item] = np.random.rand() * 2 - 1
                tmp = dataUsers[user][item] - (self.mu + self.bi[item] + self.bu[user] + self.p[user].dot(self.q[item])[0][0])
                if (optimP):
                    self.p[user] = (1 - self.lambd * self.eps) * self.p[user] + self.eps * 2 * tmp * self.q[item].transpose()
                    self.bu[user] = (1 - self.lambd * self.eps) * self.bu[user] + self.eps * 2 * tmp
                if (optimQ):
                    self.q[item] = (1 - self.lambd * self.eps) * self.q[item] + self.eps * 2 * tmp * self.p[user].transpose()
                    self.bi[item] = (1 - self.lambd * self.eps) * self.bi[item] + self.eps * 2 * tmp
                self.mu = (1 - self.lambd * self.eps) * self.mu + self.eps * 2 * tmp
                loss = loss + tmp*tmp
            self.loss.append(loss)
            if (self.alternate != 0):
                if (i % self.alternate == 0):
                    optimP = optimQ
                    optimQ = 1 - optimQ
                    print i, loss / len(couples)
            else:
                if (i % 100 == 0):
                    print i, loss / len(couples)
    def predict(self, couplesTest):
        pred = np.zeros(len(couplesTest))
        for ind,c in enumerate(couplesTest):
            pred[ind] = self.mu + self.bu[c[0]] + self.bi[c[1]] + self.p[c[0]].dot(self.q[c[1]])[0][0]
        return pred

## Factorisation de Matrices avec biais temporel

In [11]:
class matrixFactorisationBiaisTemporel():
    def __init__(self, k=10, ntimes=5, lambd=0.2, eps=1e-5, maxIter=10000, alternate=0):
        self.k = k
        self.ntimes = ntimes
        self.lambd = lambd
        self.eps = eps
        self.maxIter = maxIter
        self.alternate = alternate
    def fit(self, dataUsers, dataItems, couples, times):
        self.p = {}
        self.q = {}
        self.bu = {}
        self.bi = {}
        self.mu = np.random.rand(self.ntimes) * 2 - 1
        self.loss = []
        optimP = True
        optimQ = (self.alternate == 0)
        for i in xrange(self.maxIter):
            loss = 0
            for j in xrange(len(couples)):
                r = np.random.randint(len(couples))
                user = couples[r][0]
                item = couples[r][1]
                time = times[r]
                if not user in self.p:
                    self.p[user] = np.random.rand(1,self.k) * 2 - 1
                    self.bu[user] = np.random.rand(self.ntimes) * 2 - 1
                if not item in self.q:
                    self.q[item] = np.random.rand(self.k,1) * 2 - 1
                    self.bi[item] = np.random.rand(self.ntimes) * 2 - 1
                tmp = dataUsers[user][item] - (self.mu[time] + self.bi[item][time] + self.bu[user][time] + self.p[user].dot(self.q[item])[0][0])
                if (optimP):
                    self.p[user] = (1 - self.lambd * self.eps) * self.p[user] + self.eps * 2 * tmp * self.q[item].transpose()
                    self.bu[user][time] = (1 - self.lambd * self.eps) * self.bu[user][time] + self.eps * 2 * tmp
                if (optimQ):
                    self.q[item] = (1 - self.lambd * self.eps) * self.q[item] + self.eps * 2 * tmp * self.p[user].transpose()
                    self.bi[item][time] = (1 - self.lambd * self.eps) * self.bi[item][time] + self.eps * 2 * tmp
                self.mu[time] = (1 - self.lambd * self.eps) * self.mu[time] + self.eps * 2 * tmp
                loss = loss + tmp*tmp #Sans régularisation
            self.loss.append(loss)
            if (self.alternate != 0):
                if (i % self.alternate == 0):
                    optimP = optimQ
                    optimQ = 1 - optimQ
                    print i, loss / len(couples)
            else:
                if (i % 100 == 0):
                    print i, loss / len(couples)
    def predict(self, couplesTest, times):
        pred = np.zeros(len(couplesTest))
        for ind,c in enumerate(couplesTest):
            pred[ind] = self.mu[times[ind]] + self.bu[c[0]][times[ind]] + self.bi[c[1]][times[ind]] + self.p[c[0]].dot(self.q[c[1]])[0][0]
        return pred

# Tests les données Movie Lens 100k

## Préparation des données

In [12]:
# Chargement
data, timestamps = loadMovieLens()

# Récupérer la représentation en liste de triplets
couples = getCouplesUsersItems(data)

# La séparer en ensemble d'apprentissage et de test
trainCouples, testCouples = splitTrainTest(couples,.20)

# Reconstruire les dictionnaires pour l'ensemble d'apprentissage
trainUsers = buildUsersDict(trainCouples)
trainItems = buildItemsDict(trainCouples)

# Supprimer de l'ensemble de test les éléments inconnus en apprentissage
toDel = []
for i,c in enumerate(testCouples):
    if not c[0] in trainUsers:
        toDel.append(i)
    elif not c[1] in trainItems:
        toDel.append(i)
testCouples = np.delete(testCouples, toDel, 0)

# Reconstruire les dictionnaires pour l'ensemble de test
testUsers  = buildUsersDict(testCouples)
testItems  = buildItemsDict(testCouples)

# Récupérer les vecteurs des temps
nbins = 5
times = getTimeBins(couples, timestamps, nbins)
trainTimes = getTimeBins(trainCouples, timestamps, nbins)
testTimes = getTimeBins(testCouples, timestamps, nbins)

# taille des données
#print len(trainUsers), len(testUsers)
#print len(trainItems), len(testItems)

## Baselines

In [13]:
model1 = baselineMeanUsers()
model1.fit(trainUsers)
pred = model1.predict(testCouples)
print "erreur en test:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

erreur en test: 1.08839474696


In [14]:
model2 = baselineMeanItems()
model2.fit(trainItems)
pred = model2.predict(testCouples)
print "erreur en test:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

erreur en test: 1.03859106198


## Factorisation matricielle sans biais

In [15]:
model3 = matrixFactorisation(10, alternate=0)
model3.fit(trainUsers, trainItems, trainCouples)

0 2.829537706
100 1.30712001798
200 1.07948970774
300 0.997287439085
400 0.952657605705
500 0.917643772482
600 0.89365902144
700 0.888028692502
800 0.869048170694
900 0.868529460105
1000 0.858853615837
1100 0.863836741598
1200 0.852722256483
1300 0.842192524637
1400 0.841983968252
1500 0.844243257703
1600 0.837896616117
1700 0.837744282998
1800 0.832008944357
1900 0.83920613112


In [16]:
plt.figure()
plt.plot(model3.loss)
plt.show()

In [17]:
pred = model3.predict(testCouples)
print "Erreur de test:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

Erreur de test: 0.909580818424


## Factorisation matricielle avec biais

In [18]:
model4 = matrixFactorisationBiais(10, alternate=0)
model4.fit(trainUsers, trainItems, trainCouples)

0 7.84879153329
100 2.20841845373
200 1.78135161261
300 1.50665884754
400 1.36179780473
500 1.27271620043
600 1.18725411492
700 1.13627534893
800 1.08727524392
900 1.05073590425
1000 1.01654162391
1100 1.00560542752
1200 0.97646470343
1300 0.967124816417
1400 0.950623023452
1500 0.94110060945
1600 0.928282231649
1700 0.913576022519
1800 0.90368654631
1900 0.889919122869
2000 0.886632345112
2100 0.87637080938
2200 0.879517708617
2300 0.872767670014
2400 0.863631577612
2500 0.853651927641
2600 0.849095197608
2700 0.855620811393
2800 0.843541206755
2900 0.844477373661
3000 0.841569307323
3100 0.842851965166
3200 0.833419184501
3300 0.824305787357
3400 0.832219602661
3500 0.825525993355
3600 0.824484381541
3700 0.816336008193
3800 0.815159277787
3900 0.814879026143
4000 0.813612289994
4100 0.813574241683
4200 0.802972958589
4300 0.811059457145
4400 0.804668565967
4500 0.800480942454
4600 0.803039122674
4700 0.798049443498
4800 0.797483731818
4900 0.796329706711
5000 0.789174918466
5100 0.7

In [19]:
plt.figure()
plt.plot(model4.loss)
plt.show()

In [20]:
pred = model4.predict(testCouples)
print ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

0.90047122114


## Biais Temporel

### Visualisition des notes en fonction du temps
![alt text](./ratingsByTime1k.png "Distribution des scores")

In [21]:
ratings = np.array(np.array(couples)[:,2], float)
plt.figure()
for i in xrange(nbins):
    histi = np.bincount(np.array(ratings[times==i], int))
    plt.plot(1.* histi / histi.sum() , 'o-')
plt.show()
plt.close()

### Resultats (pas bon, a refaire)

In [21]:
model5 = matrixFactorisationBiaisTemporel(10, alternate=0)
model5.fit(trainUsers, trainItems, trainCouples, trainTimes)

0 6.67272772294
100 2.39851598656
200 2.03459234926
300 1.81139905907
400 1.66860929545
500 1.55222519475
600 1.4740954684
700 1.40879194473
800 1.36541067006
900 1.30120495766
1000 1.26982794492
1100 1.23715478364
1200 1.21515926539
1300 1.18462986766
1400 1.16283237634
1500 1.13765379581
1600 1.1211176562
1700 1.10614780981
1800 1.07874680564
1900 1.06609699254
2000 1.05368110585
2100 1.0438793476
2200 1.02470072182
2300 1.01067526082
2400 1.01651111078
2500 1.0008570694
2600 0.983412318075
2700 0.979243005145
2800 0.972186045903
2900 0.963801902146
3000 0.959951237108
3100 0.946482006126
3200 0.942638702669
3300 0.93229056953
3400 0.922386182976
3500 0.922667408037
3600 0.916931268408
3700 0.906495457662
3800 0.904733177419
3900 0.902645520406
4000 0.894787529597
4100 0.891496157402
4200 0.89578399219
4300 0.886638650815
4400 0.884556023075
4500 0.876389370103
4600 0.867495263732
4700 0.868984186507
4800 0.862418832999
4900 0.850179707736
5000 0.852969600596
5100 0.852208608488
5200

In [22]:
plt.figure()
plt.plot(model5.loss)
plt.show()

In [23]:
pred = model5.predict(testCouples, testTimes)
print ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

0.941643072909


# Experiences sur les données Movie Lens 1M

## Préparation des données

In [24]:
# Chargement
data, timestamps = loadMovieLens1M()

# Récupérer la représentation en liste de triplets
couples = getCouplesUsersItems(data)

# Séparer en ensemble d'apprentissage et de test
trainCouples, testCouples = splitTrainTest(couples,.20)

# Reconstruire les dictionnaires pour l'ensemble d'apprentissage
trainUsers = buildUsersDict(trainCouples)
trainItems = buildItemsDict(trainCouples)

# Supprimer de l'ensemble de test les éléments inconnus en apprentissage
toDel = []
for i,c in enumerate(testCouples):
    if not c[0] in trainUsers:
        toDel.append(i)
    elif not c[1] in trainItems:
        toDel.append(i)
testCouples = np.delete(testCouples, toDel, 0)

# Reconstruire les dictionnaires pour l'ensemble de test
testUsers  = buildUsersDict(testCouples)
testItems  = buildItemsDict(testCouples)

# Récupérer les vecteurs des temps
nbins = 5
times = getTimeBins(couples, timestamps, nbins)
trainTimes = getTimeBins(trainCouples, timestamps, nbins)
testTimes = getTimeBins(testCouples, timestamps, nbins)

# taille des données
#print len(trainUsers), len(testUsers)
#print len(trainItems), len(testItems)

## Baselines

In [25]:
model6 = baselineMeanUsers()
model6.fit(trainUsers)
pred = model6.predict(testCouples)
print "erreur en test:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()        

erreur en test: 1.07459570403


In [26]:
model7 = baselineMeanItems()
model7.fit(trainItems)
pred = model7.predict(testCouples)
print "erreur en test:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()        

erreur en test: 0.958876325654


## Factorisation Matricielle

In [27]:
model8 = matrixFactorisation(10, alternate=0, maxIter=1000)
model8.fit(trainUsers, trainItems, trainCouples)

0 2.85630500372
100 0.989208053185
200 0.894268607039
300 0.859435186046
400 0.843744226415
500 0.834776300235
600 0.829587454159
700 0.826814746078
800 0.820488518828
900 0.820793261528


In [28]:
plt.figure()
plt.plot(model8.loss)
plt.show()

In [29]:
pred = model8.predict(testCouples)
print ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

0.848198539785


## Factorisation Matricielle avec biais

In [30]:
model9 = matrixFactorisationBiais(10, alternate=0, maxIter=2000)
model9.fit(trainUsers, trainItems, trainCouples)

0 3.47218672534
100 1.53995091541
200 1.22517433171
300 1.09549293316
400 1.02389512199
500 0.978845337117
600 0.94824016809
700 0.925148099365
800 0.90515288607
900 0.894416953748
1000 0.88204288694
1100 0.871257580652
1200 0.864077916237
1300 0.858869586043
1400 0.852355995456
1500 0.847843742846
1600 0.841015305358
1700 0.836168926899
1800 0.835827627137
1900 0.832978355848


In [31]:
pred = model9.predict(testCouples)
print ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

0.880414019261


In [32]:
plt.figure()
plt.plot(model9.loss)
plt.show()

## Factorisation de Matrices avec biais temporel

In [33]:
model10 = matrixFactorisationBiaisTemporel(10, alternate=0, maxIter=2000)
model10.fit(trainUsers, trainItems, trainCouples, trainTimes)

In [ ]:
plt.figure()
plt.plot(model10.loss)
plt.show()

In [ ]:
pred = model10.predict(testCouples, testTimes)
print ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

# Conclusion